# Books EDA and recommendation system

Finding a dataset with reasonnable size and data about user is not common. It's why I have chosen the data at https://www.kaggle.com/datasets/saurabhbagchi/books-dataset.
A said in the Acknowledgements part of this page, the data was collected by Cai-Nicolas Ziegler in 2004. The original links is http://www2.informatik.uni-freiburg.de/~cziegler/BX/.
The data is a litle old but it provides a good dataset for training. The usability score on Kaggle is 10.0 which is fairly good. Other datasets with user ratings were too voluminous like this one https://www.kaggle.com/datasets/bahramjannesarr/goodreads-book-datasets-10m, the usability was bad or the source was not provided.

This notebook is divided in two parts. The first is an EDA about the books and teh second is a recommendations system. 

## Setup the environment

An environment was created for this project. The python version is 3.10.8 as seen below

In [1]:
!echo %CONDA_DEFAULT_ENV%

mlProject


In [14]:
!python --version

Python 3.10.8


In [15]:
!pip show pandas

Name: pandas
Version: 1.5.2
Summary: Powerful data structures for data analysis, time series, and statistics
Home-page: https://pandas.pydata.org
Author: The Pandas Development Team
Author-email: pandas-dev@python.org
License: BSD-3-Clause
Location: c:\users\adrie\anaconda3\envs\mlproject\lib\site-packages
Requires: numpy, python-dateutil, pytz
Required-by: seaborn


To use linear algebra we import numpy and to process the data we import pandas

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing

To get and manipulate file, we import the os package

In [2]:
import os

To visualize data we install an import matplotlib and to improv visualization we use seaborn

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Loading the data

In [4]:
#path and file name
files_path="D:/Workspace_Python/MLProject/dataset/books_data/"
books_csv="books.csv"
users_csv="users.csv"
ratings_csv="ratings.csv"

In [5]:
books = pd.read_csv(files_path+books_csv, sep=";",on_bad_lines='warn', encoding="latin-1")
books.columns = ['ISBN', 'bookTitle', 'bookAuthor', 'yearOfPublication', 'publisher', 
                 'imageUrlS', 'imageUrlM', 'imageUrlL']

C:\Users\adrie\AppData\Local\Temp\ipykernel_16336\4244541532.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv(files_path+books_csv, sep=";",on_bad_lines='warn', encoding="latin-1")


As we can see, some lines were skipped because they have too many fields. We should see if this data can be cleanned so we can used them. As we doesn't have many rows in error and the files are readable with tools like Notepad++ or Excel, we can do it manually. If it wasn't the case we should have wrote a custom reader to store the line in error.
When looking at the file we can see the presence of "&amp;" string which can create unexpected columns du to the semicolon. To correct that, we remplace "&amp;"  by "&"
By rerunning the read_csv function, we still have some errors:

Skipping line 43667: expected 8 fields, saw 10

Skipping line 144058: expected 8 fields, saw 9
Skipping line 180189: expected 8 fields, saw 9

Skipping line 209388: expected 8 fields, saw 9
Skipping line 227933: expected 8 fields, saw 10
Skipping line 261529: expected 8 fields, saw 9

Again this result of title containing both " \" " and " ; ". As we have only 6 rows affected, we correct them manually by replacing " \" " by " ' " then " ; " by " , " if necessary.
Rerunning the above cell and check if all lines are loaded:

In [42]:
books.shape

(271379, 8)

The number of rows and columns in the dataFram is the same than the file, so far so good.
We can now explore the data

## Search of inconsistency in the data

### Cleaning of the books dataset

In [32]:
books.head()

,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher,imageUrlS,imageUrlM,imageUrlL
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton & Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


We observe that the three last columns are URL of image so we can discard them as they don't contain useful data. 

In [6]:
books=books.drop(['imageUrlS','imageUrlM','imageUrlL'], axis=1)
books.head()

,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton & Company


The column yearOfPublication should contain only year and be convertible to integer. We check if it's the case

In [7]:
books["yearOfPublication"].unique()

array([2002, 2001, 1991, 1999, 2000, 1993, 1996, 1988, 2004, 1998, 1994,
       2003, 1997, 1983, 1979, 1995, 1982, 1985, 1992, 1986, 1978, 1980,
       1952, 1987, 1990, 1981, 1989, 1984, 0, 1968, 1961, 1958, 1974,
       1976, 1971, 1977, 1975, 1965, 1941, 1970, 1962, 1973, 1972, 1960,
       1966, 1920, 1956, 1959, 1953, 1951, 1942, 1963, 1964, 1969, 1954,
       1950, 1967, 2005, 1957, 1940, 1937, 1955, 1946, 1936, 1930, 2011,
       1925, 1948, 1943, 1947, 1945, 1923, 2020, 1939, 1926, 1938, 2030,
       1911, 1904, 1949, 1932, 1928, 1929, 1927, 1931, 1914, 2050, 1934,
       1910, 1933, 1902, 1924, 1921, 1900, 2038, 2026, 1944, 1917, 1901,
       2010, 1908, 1906, 1935, 1806, 2021, '1999', '2002', '2001', '2000',
       '1996', '1992', '1995', '2004', '2003', '1990', '1994', '1986',
       '1989', '1981', '1993', '1983', '1982', '1976', '1991', '1977',
       '1998', '0', '1997', '1974', '1968', '1987', '1984', '1988',
       '1963', '1956', '1970', '1985', '1978', '1973', '1980'

We observe that some date are instead publishers. We retrieve this observations:

In [24]:
bad_rows=books[(books["yearOfPublication"]=='DK Publishing Inc') | (books["yearOfPublication"] == 'Gallimard')]
bad_rows

,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher
209550,078946697X,"DK Readers: Creating the X-Men, How It All Beg...",2000,DK Publishing Inc,http://images.amazon.com/images/P/078946697X.0...
220744,2070426769,"Peuple du ciel, suivi de 'Les Bergers\"";Jean-M...",2003,Gallimard,http://images.amazon.com/images/P/2070426769.0...
221691,0789466953,"DK Readers: Creating the X-Men, How Comic Book...",2000,DK Publishing Inc,http://images.amazon.com/images/P/0789466953.0...


The book title seems to contain the book author. We clean this rows.

In [25]:
bad_rows[['realBookTitle','realBookAuthor']]=bad_rows.bookTitle.str.split('\\\\";',expand=True)
bad_rows

C:\Users\adrie\AppData\Local\Temp\ipykernel_16336\1544933059.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bad_rows[['realBookTitle','realBookAuthor']]=bad_rows.bookTitle.str.split('\\\\";',expand=True)
C:\Users\adrie\AppData\Local\Temp\ipykernel_16336\1544933059.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bad_rows[['realBookTitle','realBookAuthor']]=bad_rows.bookTitle.str.split('\\\\";',expand=True)


,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher,realBookTitle,realBookAuthor
209550,078946697X,"DK Readers: Creating the X-Men, How It All Beg...",2000,DK Publishing Inc,http://images.amazon.com/images/P/078946697X.0...,"DK Readers: Creating the X-Men, How It All Beg...","Michael Teitelbaum"""
220744,2070426769,"Peuple du ciel, suivi de 'Les Bergers\"";Jean-M...",2003,Gallimard,http://images.amazon.com/images/P/2070426769.0...,"Peuple du ciel, suivi de 'Les Bergers","Jean-Marie Gustave Le ClÃ?Â©zio"""
221691,0789466953,"DK Readers: Creating the X-Men, How Comic Book...",2000,DK Publishing Inc,http://images.amazon.com/images/P/0789466953.0...,"DK Readers: Creating the X-Men, How Comic Book...","James Buckley"""


In [26]:
lst=[]
for i in bad_rows.index:
    lst.append(i)
    books.loc[i,"publisher"]=bad_rows.loc[i,"yearOfPublication"]
    books.loc[i,"yearOfPublication"]=bad_rows.loc[i,"bookAuthor"]
    books.loc[i,"bookAuthor"]=bad_rows.loc[i,"realBookAuthor"]
    books.loc[i,"bookTitle"]=bad_rows.loc[i,"realBookTitle"]
books.loc[lst]

,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher
209550,078946697X,"DK Readers: Creating the X-Men, How It All Beg...","Michael Teitelbaum""",2000,DK Publishing Inc
220744,2070426769,"Peuple du ciel, suivi de 'Les Bergers","Jean-Marie Gustave Le ClÃ?Â©zio""",2003,Gallimard
221691,0789466953,"DK Readers: Creating the X-Men, How Comic Book...","James Buckley""",2000,DK Publishing Inc


In [27]:
books["bookAuthor"]=books["bookAuthor"].str.replace('"',"'")

In [28]:
books["bookAuthor"]=books["bookAuthor"].str.replace(r'\\',"")

C:\Users\adrie\AppData\Local\Temp\ipykernel_16336\1141391955.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  books["bookAuthor"]=books["bookAuthor"].str.replace(r'\\',"")


Some rows has quote or double quote at the end of bookAuthor or bookTitle that doesn't close another quote. We remove them.

In [29]:
books[books["bookAuthor"].str.endswith("'",na=False) & books["bookAuthor"].str.count("'")%2==1]

,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher
98288,0416652700,The Pooh cook book: inspired by \Winnie-the-Po...,;Katie Stewart',1971,Methuen
112280,1853114103,Lord Have Mercy,'Ron'',2001,Canterbury Press
117995,0786881364,A Simple Program : A Contemporary Translation ...,J 'j'',1996,Hyperion
209550,078946697X,"DK Readers: Creating the X-Men, How It All Beg...",Michael Teitelbaum',2000,DK Publishing Inc
220744,2070426769,"Peuple du ciel, suivi de 'Les Bergers",Jean-Marie Gustave Le ClÃ?Â©zio',2003,Gallimard
221691,0789466953,"DK Readers: Creating the X-Men, How Comic Book...",James Buckley',2000,DK Publishing Inc
223761,1895455243,Finger Food (Company's Coming Pint Size),Jean Pare',1993,Company's Coming Publishing
245727,0961364920,A Timeless Place: Lett's Set a Spell at the Co...,Sandra 'Sandy Lynn' Lett',2001,Transformations


In [30]:
condition=books["bookAuthor"].str.endswith("'",na=False) & books["bookAuthor"].str.count("'")%2==1
books.loc[condition,"bookAuthor"]=books.loc[condition,"bookAuthor"].str.rstrip("'")

In [31]:
books[books["bookAuthor"].str.endswith('"',na=False) & books["bookAuthor"].str.count('"')%2==1]

,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher


In [35]:
condition=books["bookTitle"].str.endswith("'",na=False) & books["bookTitle"].str.count("'")%2==1
books.loc[condition,"bookTitle"]

Series([], Name: bookTitle, dtype: object)

In [33]:
books.loc[condition,"bookTitle"]=books.loc[condition,"bookTitle"].str.rstrip("'")

In [36]:
books.loc[condition,"bookTitle"]

Series([], Name: bookTitle, dtype: object)

In [37]:
condition=books["bookTitle"].str.endswith('"',na=False) & books["bookTitle"].str.count('"')%2==1
books.loc[condition,"bookTitle"]=books.loc[condition,"bookTitle"].str.rstrip("'")

In [38]:
books["bookTitle"]=books["bookTitle"].str.replace(r'\\',"")

C:\Users\adrie\AppData\Local\Temp\ipykernel_16336\3008223896.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  books["bookTitle"]=books["bookTitle"].str.replace(r'\\',"")


We transform the year in integer to better handle them

In [39]:
books.yearOfPublication.unique()

array([2002, 2001, 1991, 1999, 2000, 1993, 1996, 1988, 2004, 1998, 1994,
       2003, 1997, 1983, 1979, 1995, 1982, 1985, 1992, 1986, 1978, 1980,
       1952, 1987, 1990, 1981, 1989, 1984, 0, 1968, 1961, 1958, 1974,
       1976, 1971, 1977, 1975, 1965, 1941, 1970, 1962, 1973, 1972, 1960,
       1966, 1920, 1956, 1959, 1953, 1951, 1942, 1963, 1964, 1969, 1954,
       1950, 1967, 2005, 1957, 1940, 1937, 1955, 1946, 1936, 1930, 2011,
       1925, 1948, 1943, 1947, 1945, 1923, 2020, 1939, 1926, 1938, 2030,
       1911, 1904, 1949, 1932, 1928, 1929, 1927, 1931, 1914, 2050, 1934,
       1910, 1933, 1902, 1924, 1921, 1900, 2038, 2026, 1944, 1917, 1901,
       2010, 1908, 1906, 1935, 1806, 2021, '1999', '2002', '2001', '2000',
       '1996', '1992', '1995', '2004', '2003', '1990', '1994', '1986',
       '1989', '1981', '1993', '1983', '1982', '1976', '1991', '1977',
       '1998', '0', '1997', '1974', '1968', '1987', '1984', '1988',
       '1963', '1956', '1970', '1985', '1978', '1973', '1980'

In [40]:
books.yearOfPublication=books.yearOfPublication.astype("int")

We check the presence of null value in the dataFrame

In [41]:
books[books.yearOfPublication.isna()]

,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher


In [42]:
books[books.bookTitle.isna()]

,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher


In [43]:
books[books.bookAuthor.isna()]

,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher
187700,9627982032,The Credit Suisse Guide to Managing Your Perso...,NaN,1995,Edinburgh Financial Publishing


In [44]:
books[books.publisher.isna()]

,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher
128896,193169656X,Tyrant Moon,Elaine Corvidae,2002,NaN
129043,1931696993,Finders Keepers,Linnea Sinclair,2001,NaN


In [45]:
books[books.ISBN.isna()]

,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher


Only the columns bookAuthor and publisher have null value and very few. We decide to keep them as the information in these rows can still be useful. We always could delete them later.

### Cleaning of the users dataset

In [67]:
users = pd.read_csv(files_path+users_csv, sep=';', on_bad_lines='warn', encoding="latin-1")
users.columns = ['userID', 'Location', 'Age']

In [60]:
users[ (users.Age>118) | (users.Age<5) ].count()[0] 
#118 is the age of the oldest person nowadays, 5 is the regular age where children learn to read

961

They are 960 users which have incoherent or irrelevant age. We update this values with nan value

In [68]:
users.loc[ (users.Age>118) | (users.Age<5), 'Age'] = np.nan

In [49]:
users.dtypes

userID        int64
Location     object
Age         float64
dtype: object

The column types are good so we change nothing.

In [50]:
users[users.Location.str.contains('"')]

,userID,Location,Age
267,268,"århus, \n/a\"", denmark""",NaN
1102,1103,"kuala lumpur, \n/a\"", malaysia""",NaN
1272,1273,"valladolid, \n/a\"", spain""",27.0
1305,1306,"lisbon, \n/a\"",""",41.0
1694,1695,"london, \n/a\"", united kingdom""",69.0
...,...,...,...
277747,277748,"rimini, \n/a\"", italy""",NaN
277859,277860,"barcelona, \n/a\"", spain""",25.0
278137,278138,"oviedo, \n/a\"", spain""",49.0
278342,278343,"växjö, \n/a\"", sweden""",NaN


We can do the same operation done for books

In [69]:
users["Location"]=users["Location"].str.replace('"',"'")
users["Location"]=users["Location"].str.replace(r'\\',"")

C:\Users\adrie\AppData\Local\Temp\ipykernel_16336\1594432378.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  users["Location"]=users["Location"].str.replace(r'\\',"")


In [79]:
users["Location"]=users["Location"].str.replace("n/a'","n/a")
users["Location"]=users["Location"].str.replace("'n/a","n/a")

In [81]:
condition=users["Location"].str.endswith("'",na=False) & users["Location"].str.count("'")%2==1
users.loc[condition,"Location"]=users.loc[condition,"Location"].str.rstrip("'")

In [84]:
users[users.Location.str.contains("'")].sort_values(by="Location")

,userID,Location,Age
244250,244251,"alexandria'., 'alexandria'., egypt'",44.0
80169,80170,"athens, ano liosia', greece'",16.0
10276,10277,"düsseldorf, na', germany'",NaN
158161,158162,"holbæk, n/j', denmark'",NaN
119745,119746,"kartoum, n/s', sudan'",28.0
69341,69342,"kocaeli, na'., turkey'",NaN
197600,197601,"ljubljana, na', slovenia'",NaN
72128,72129,"lleida, catalunya', spain'",21.0
72592,72593,"lujan de cuyo, mendoza/argentina', argentina'",25.0
252668,252669,"manchester, na', united kingdom'",NaN


We observe we can remove all the remaining quote in the column Location as all are irrelevant.

In [85]:
users["Location"]=users["Location"].str.replace("'","")

In [86]:
users[users.Location.str.contains("'")]

,userID,Location,Age


The users datafram is now clean, we apply the same process to the ratings dataset

### Cleaning the ratings dataset

In [179]:
ratings = pd.read_csv(files_path+ratings_csv, sep=';',  on_bad_lines='warn', encoding="latin-1")
ratings.columns = ['userID', 'ISBN', 'bookRating']

In [88]:
ratings.head()

,userID,ISBN,bookRating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [90]:
ratings.bookRating.unique()

array([ 0,  5,  3,  6,  8,  7, 10,  9,  4,  1,  2], dtype=int64)

The bookRating column has value between 0 and 10 as expected so nothing to do with this column

In [115]:
[print("userID as na value") if ratings[ratings[x].isna()].shape[0]!=0 else print(f"all good for {x}")  for x in ratings.columns] 

all good for userID
all good for ISBN
all good for bookRating


[None, None, None]

None of the columns have na value

In [117]:
ratings.dtypes

userID         int64
ISBN          object
bookRating     int64
dtype: object

All userID are integers so they aren't format problem for this feature

In [180]:
condition=(ratings["ISBN"].str.contains('"') ) | (ratings["ISBN"].str.contains(r'\\') ) 
bad_isbn=ratings.loc[condition]
bad_isbn

,userID,ISBN,bookRating
21687,4377,"\0210000010""",8
45327,11676,"\0432534220\""""",6
45328,11676,"\2842053052\""""",7
183260,39821,"\0432534220\""""",6
210091,48821,"\8804501367\""""",0
531936,128375,"\2842053052\""""",0
663235,160867,"\0094749809""",7
824820,199138,"\0432534220\""""",8
877347,212647,"\8804520159\""""",7
941677,227945,"\9170010242\""""",10


As for the previous dataset, we observe error of quoting on some observation. They still represente accurate data hence we choose to clean them in orer to keep them.

In [181]:
bad_isbn["ISBN"]=bad_isbn["ISBN"].str.replace('"',"")
bad_isbn["ISBN"]=bad_isbn["ISBN"].str.replace('\\',"",regex=True)
bad_isbn

C:\Users\adrie\AppData\Local\Temp\ipykernel_16336\2125878562.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bad_isbn["ISBN"]=bad_isbn["ISBN"].str.replace('"',"")
C:\Users\adrie\AppData\Local\Temp\ipykernel_16336\2125878562.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bad_isbn["ISBN"]=bad_isbn["ISBN"].str.replace('\\',"",regex=True)


,userID,ISBN,bookRating
21687,4377,0210000010,8
45327,11676,0432534220,6
45328,11676,2842053052,7
183260,39821,0432534220,6
210091,48821,8804501367,0
531936,128375,2842053052,0
663235,160867,0094749809,7
824820,199138,0432534220,8
877347,212647,8804520159,7
941677,227945,9170010242,10


In [182]:
ratings.update(bad_isbn)

In [184]:
condition=(ratings["ISBN"].str.contains('"') ) | (ratings["ISBN"].str.contains(r'\\') ) 
ratings[condition]

,userID,ISBN,bookRating


All data are now cleaned. We pass to the analysis part.

## EDA

## Recommendation system